# #11a Joint Elements (Points)
<i>Create joints between two separate points</i>
***

### Joints for points

In [1]:
import sys; sys.path.append('../') # Reference modules in parent directory

from LPI import *
lusas = get_lusas_modeller()

if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please close the current model first")

import m100_Tools_And_Helpers.Helpers as Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Joint for points.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")
lusas.view().setIsometric()
lusas.view().showCoordSystem(False)

db.createLoadcase("")
db.createLoadcase("")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFLoadcase instance at 0x2489449731472>

### Supports

In [2]:
support_attr = db.createSupportStructural("Sup1")
support_attr.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "C", "F")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFSupportStructural instance at 0x2489449740112>

In [3]:
point_load_x_attr = db.createLoadingConcentrated("Point Load X").setConcentrated(1,0,0,0,0,0)
point_load_y_attr = db.createLoadingConcentrated("Point Load Y").setConcentrated(0,1,0,0,0,0)
point_load_z_attr = db.createLoadingConcentrated("Point Load Z").setConcentrated(0,0,1,0,0,0)
loads = [point_load_x_attr, point_load_y_attr, point_load_z_attr]

### Joint mesh attributes

In [4]:
# Joints assigned between two separate features
mesh_pinned = db.createPointMeshElementAttr("Point Mesh Pinned", "JNT4")
mesh_moment = db.createPointMeshElementAttr("Point Mesh Moments", "JSH4")

### Joint Materials

In [5]:
pinned_material_attr = db.createSpringJointMaterial("Pinned Material", [100, 100, 100]).setValue("Assignment", "Point")
moment_material_attr = db.createSpringJointMaterial("Moments material", [100, 100, 100, 100, 100, 100]).setValue("Assignment", "Point")

### Joint Geometry

In [6]:
# Analysis 1 has no joint geometry assigned. In this case the nodes of the joint should be coincident, no eccentricity (lever arm) is considered.

# Analysis 2, Use joint length to determine the eccentricity.
geom_by_length_attr = db.createGeometricJoint("Geom (joint length = 1)").setJoint(0.0, 0.5, 0.5, True)

# Analysis 3, specify a known eccentricty of 0.5, regardless of joint length/arrangement.
geom_ecc_z_attr     = db.createGeometricJoint("Geom (eccentricity = 0.5)").setJoint(0.5, 0.0, 0.0, False)

### Joints between two points

In [7]:
lower_points:list[IFPoint]=[]
upper_points:list[IFPoint]=[]

for x in [0, 1]:
    lower_points.append(Helpers.create_point(x, 0, 0))
    upper_points.append(Helpers.create_point(x, 0, 1))

In [8]:
support_attr.assignTo(lower_points)
pinned_material_attr.assignTo(lower_points[0])
moment_material_attr.assignTo(lower_points[1])

mesh_pinned.assignTo([lower_points[0], upper_points[0]])
mesh_moment.assignTo([lower_points[1], upper_points[1]])

point_load_x_attr.assignTo(upper_points)
assign = lusas.assignment().setAllDefaults().setLoadset(2)
point_load_y_attr.assignTo(upper_points, assign)
assign = lusas.assignment().setAllDefaults().setLoadset(3)
point_load_z_attr.assignTo(upper_points, assign)


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x2489439859200>

### Eccentric joints

In [9]:
for i in range(2):
    analysis = db.createAnalysisStructural("", False)

    for j in range(3):
        loadcase = db.createLoadcase("", analysis.getName())
        assign = lusas.assignment().setAllDefaults().setLoadset(loadcase)
        loads[j].assignTo(upper_points, assign)

        if i == 0:
            geom_by_length_attr.assignTo(lower_points, assign)
        else:
            geom_ecc_z_attr.assignTo(lower_points, assign)


### Solve

In [10]:
db.resetMesh()
db.updateMesh()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFDatabase instance at 0x2489451402096>

In [11]:
for analysis in db.getAnalyses():
   analysis.solve(True)
db.openAllResults(True, True)

lusas.view().insertValuesLayer()
lusas.view().values().setResults("Force/Moment - 3D Joint (JSH4,JL46)", "My")
lusas.view().values().setShowAtGauss()
lusas.view().mesh().showElementOrient(True)

### Display Reactions for each loadcase

In [12]:
prw = db.createPrintResultsWizard("Reactions per loadcase")
prw.setResultsOrder("Feature")
prw.setResultsContent("Tabular")
prw.setResultsEntity("Reaction")
prw.setResultsLocation("Nodal")
prw.setLoadcasesOption("All")
prw.setComponents(["All"])
prw.setDisplayNow(True)
prw.showResults()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFGridWindow instance at 0x2489450058416>